In [86]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from typing import List, Tuple
import time
import json
import os


class OperatorAssistant:
    def __init__(self, model_name: str = './paraphrase-multilingual-MiniLM-L12-v2'):
        """
        Инициализация помощника оператора с моделью эмбеддингов
        """
        self.model = SentenceTransformer(model_name)
        self.knowledge_base = []
        self.embeddings = None
        self.index = None
        self.dimension = 0

    def initialize_knowledge_base(self, phrases: List[str]) -> None:
        """
        Инициализация базы знаний и создание FAISS индекса
        """
        self.knowledge_base = phrases
        if not phrases:
            raise ValueError("База знаний не может быть пустой")

        # Кодирование фраз в эмбеддинги
        start_time = time.time()
        self.embeddings = self.model.encode(phrases, show_progress_bar=True)
        self.embeddings = np.array(self.embeddings)
        self.dimension = self.embeddings.shape[1]

        # Создание и заполнение индекса FAISS
        self.index = faiss.IndexFlatL2(self.dimension)
        self.index.add(self.embeddings)

        print(f"✅ База знаний инициализирована: {len(phrases)} фраз, "
              f"размерность {self.dimension}, время обработки: {time.time() - start_time:.2f} сек")

    def find_similar_responses(self, query: str, top_k: int = 3) -> Tuple[List[str], List[float]]:
        """
        Поиск наиболее подходящих ответов на запрос клиента
        Возвращает список фраз и соответствующие scores (схожести)
        """
        if self.index is None or self.embeddings is None:
            raise RuntimeError("База знаний не инициализирована")

        if not query.strip():
            raise ValueError("Запрос не может быть пустым")

        # Кодирование запроса
        query_embedding = self.model.encode([query])
        query_embedding = np.array(query_embedding)

        # Поиск в индексе
        distances, indices = self.index.search(query_embedding, top_k)

        # Преобразование расстояний в схожесть
        responses = [self.knowledge_base[i] for i in indices[0]]
        similarities = [1 - (dist / (1 + dist)) for dist in distances[0]]

        return responses, similarities

    def add_to_knowledge_base(self, new_phrases: List[str]) -> None:
        """
        Добавление новых фраз в базу знаний с обновлением индекса
        """
        if not new_phrases:
            return

        # Кодирование новых фраз
        new_embeddings = self.model.encode(new_phrases)
        new_embeddings = np.array(new_embeddings)

        # Проверка размерности
        if self.index is None:
            self.dimension = new_embeddings.shape[1]
            self.index = faiss.IndexFlatL2(self.dimension)
            self.embeddings = new_embeddings
        else:
            if new_embeddings.shape[1] != self.dimension:
                raise ValueError("Размерность новых эмбеддингов не совпадает с существующей")
            self.embeddings = np.vstack([self.embeddings, new_embeddings])
            self.index.add(new_embeddings)

        self.knowledge_base.extend(new_phrases)
        print(f"✅ Добавлено {len(new_phrases)} новых фраз в базу знаний")

    def save_knowledge_base(self, save_dir: str = "knowledge_base") -> None:
        """
        Сохраняет базу знаний, эмбеддинги и индекс в указанную директорию
        """
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Сохраняем фразы
        with open(os.path.join(save_dir, "phrases.json"), "w", encoding="utf-8") as f:
            json.dump(self.knowledge_base, f, ensure_ascii=False, indent=2)

        # Сохраняем эмбеддинги
        if self.embeddings is not None:
            np.save(os.path.join(save_dir, "embeddings.npy"), self.embeddings)

        # Сохраняем FAISS индекс
        if self.index is not None:
            faiss.write_index(self.index, os.path.join(save_dir, "index.faiss"))

        print(f"✅ База знаний сохранена в директорию: {save_dir}")

    def load_knowledge_base(self, save_dir: str = "knowledge_base") -> bool:
        """
        Загружает базу знаний, эмбеддинги и индекс из указанной директории
        """
        phrases_path = os.path.join(save_dir, "phrases.json")
        embeddings_path = os.path.join(save_dir, "embeddings.npy")
        index_path = os.path.join(save_dir, "index.faiss")

        if not os.path.exists(phrases_path) or not os.path.exists(embeddings_path) or not os.path.exists(index_path):
            print(f"⚠ Файлы базы знаний не найдены в {save_dir}")
            return False

        try:
            # Загружаем фразы
            with open(phrases_path, "r", encoding="utf-8") as f:
                self.knowledge_base = json.load(f)

            # Загружаем эмбеддинги
            self.embeddings = np.load(embeddings_path)
            self.dimension = self.embeddings.shape[1]

            # Загружаем индекс
            self.index = faiss.read_index(index_path)

            print(f"✅ База знаний загружена: {len(self.knowledge_base)} фраз, "
                  f"размерность {self.dimension}")
            return True
        except Exception as e:
            print(f"⚠ Ошибка при загрузке базы знаний: {str(e)}")
            return False

        
        
# Пример использования
if __name__ == "__main__":
    # Инициализация базы знаний
    knowledge_base = [
    # Приветствия и стандартные фразы
    "Добрый день! Чем могу помочь?",
    "Здравствуйте! Слушаю вас.",
    "Рады вас слышать! Какой у вас вопрос?",
    "Приветствуем в нашем сервисе! Чем могу быть полезен?",
    
    # Уточняющие вопросы
    "Пожалуйста, уточните номер договора.",
    "Для помощи мне нужно узнать ваш идентификатор клиента.",
    "Не могли бы вы уточнить детали вашего запроса?",
    "Какой именно сервис вас интересует?",
    "Вы обращаетесь по вопросу кредита, депозита или другого продукта?",
    
    # Оплата и платежи
    "Вы можете оплатить через мобильное приложение или терминал.",
    "Для оплаты доступны онлайн-банкинг, мобильное приложение и платежные терминалы.",
    "Оплатить можно через личный кабинет на нашем сайте.",
    "Реквизиты для оплаты будут отправлены вам в SMS.",
    "Автоплатеж можно настроить в личном кабинете.",
    "При оплате через терминал комиссия не взимается.",
    
    # Кредиты и задолженности
    "Мы можем предложить вам реструктуризацию задолженности.",
    "Информацию о текущей задолженности можно узнать в личном кабинете.",
    "График платежей был отправлен вам на email при оформлении кредита.",
    "Вы можете подать заявку на увеличение кредитного лимита.",
    "Досрочное погашение кредита доступно без комиссий.",
    
    # Безопасность и верификация
    "Для безопасности мне нужно провести верификацию.",
    "Ожидайте СМС с кодом подтверждения.",
    "К сожалению, я не могу разглашать эту информацию без верификации.",
    "Пожалуйста, назовите кодовое слово из договора.",
    "Доступ к этой информации возможен только после подтверждения личности.",
    
    # Технические проблемы
    "Приносим извинения за технические неполадки. Мы уже работаем над решением.",
    "Попробуйте обновить страницу или зайти позже.",
    "Для решения проблемы попробуйте очистить кэш браузера.",
    "Наши технические специалисты уже в курсе проблемы.",
    
    # Переключение на специалиста
    "Перевожу вас на специалиста по данному вопросу.",
    "Мой коллега лучше поможет вам с этим вопросом.",
    "Соединяю с отделом по работе с клиентами.",
    "Этот вопрос требует уточнения, перевожу на профильного специалиста.",
    
    # Завершение разговора
    "Спасибо за обращение, хорошего дня!",
    "Благодарим за ваш звонок!",
    "Если у вас будут еще вопросы - мы всегда на связи!",
    "Желаем вам приятного дня!",
    "Всего доброго!",
    
    # Информация о продуктах
    "У нас есть специальные условия для новых клиентов.",
    "Подробнее о тарифах можно узнать на нашем сайте.",
    "Актуальные акции отражены в личном кабинете.",
    "Вы можете оформить продукт онлайн без визита в офис.",
    
    # Работа с претензиями
    "Ваша жалоба зарегистрирована под номером #XXXX.",
    "Мы рассмотрим ваше обращение в течение 3 рабочих дней.",
    "Приносим извинения за доставленные неудобства.",
    "Ваше обращение будет передано в службу контроля качества.",
    
    # Карты и счета
    "Информация о вашей карте доступна в мобильном приложении.",
    "Вы можете заказать перевыпуск карты в личном кабинете.",
    "Лимиты по карте можно увеличить, предоставив справку о доходах.",
    "Выписку по счету можно получить в отделении или онлайн.",
    
    # Мобильное приложение
    "Мобильное приложение доступно в AppStore и Google Play.",
    "В приложении доступны все основные операции.",
    "Вы можете настроить уведомления в настройках приложения.",
    "Для входа в приложение используйте те же данные, что и в личном кабинете.",
    
    # Валютные операции
    "Актуальные курсы валют отображаются в личном кабинете.",
    "Вы можете зарезервировать валюту онлайн и забрать в отделении.",
    "Лимит на валютные операции составляет $10,000 в месяц.",
    
    # Страховые продукты
    "Мы можем предложить вам несколько вариантов страхования.",
    "Страховой полис можно оформить онлайн за 5 минут.",
    "У нас есть специальные условия по страхованию для держателей карт.",
    
    # Инвестиции
    "Наши консультанты помогут вам подобрать инвестиционный продукт.",
    "Вы можете начать инвестировать с суммы от 1000 рублей.",
    "Доходность за прошлый год составила 7.2% годовых."
]
    
    # Создание помощника
    assistant = OperatorAssistant()
    assistant.initialize_knowledge_base(knowledge_base)
    


Batches: 100%|██████████| 2/2 [00:00<00:00,  4.53it/s]

✅ База знаний инициализирована: 63 фраз, размерность 384, время обработки: 0.45 сек


In [87]:
    # Пример запроса
queries = [
        "Как я могу оплатить кредит?",
        "Мне нужна помощь с договором",
        "Что делать если пришло смс?",
        "Добрый день, хочу реструктуризировать долг"
    ]
    
for query in queries:
    print(f"\n👨 Клиент: '{query}'")
    responses, similarities = assistant.find_similar_responses(query)
        
    print("🔍 Подходящие подсказки оператору:")
    for i, (resp, sim) in enumerate(zip(responses, similarities), 1):
        print(f"{i}. [{sim:.2f}] {resp}")


👨 Клиент: 'Как я могу оплатить кредит?'
🔍 Подходящие подсказки оператору:
1. [0.06] График платежей был отправлен вам на email при оформлении кредита.
2. [0.06] Досрочное погашение кредита доступно без комиссий.
3. [0.05] Вы обращаетесь по вопросу кредита, депозита или другого продукта?

👨 Клиент: 'Мне нужна помощь с договором'
🔍 Подходящие подсказки оператору:
1. [0.11] Пожалуйста, назовите кодовое слово из договора.
2. [0.08] Пожалуйста, уточните номер договора.
3. [0.08] Приветствуем в нашем сервисе! Чем могу быть полезен?

👨 Клиент: 'Что делать если пришло смс?'
🔍 Подходящие подсказки оператору:
1. [0.07] Реквизиты для оплаты будут отправлены вам в SMS.
2. [0.07] Ожидайте СМС с кодом подтверждения.
3. [0.05] Приветствуем в нашем сервисе! Чем могу быть полезен?

👨 Клиент: 'Добрый день, хочу реструктуризировать долг'
🔍 Подходящие подсказки оператору:
1. [0.15] Мы можем предложить вам реструктуризацию задолженности.
2. [0.07] Информацию о текущей задолженности можно узнать в личном к

In [73]:
assistant = OperatorAssistant()
assistant.initialize_knowledge_base(knowledge_base)  # Инициализируем начальной базой

# Новые фразы для добавления
new_phrases = [
    # Новые приветствия
    "Доброго времени суток! Чем вам помочь?",
    "Привет! Я ваш виртуальный помощник, задавайте вопрос.",
    
    # Новые варианты по платежам
    "Вы можете настроить автоплатеж в разделе 'Регулярные платежи' мобильного приложения.",
    "При оплате через наш сайт действует cashback 1%.",
    
    # Новые сценарии
    "Для восстановления доступа к личному кабинету нажмите 'Забыли пароль' на странице входа.",
    "Вы можете заказать выписку по email в настройках профиля.",
    "Для оформления кредитной карты потребуется паспорт и ИНН.",
    
    # Ответы на частые жалобы
    "Приносим извинения за долгое ожидание, ваш запрос действительно важен для нас.",
    "Мы ценим ваше терпение и обязательно решим ваш вопрос."
]

# Добавляем новые фразы в базу знаний
assistant.add_to_knowledge_base(new_phrases)

# Проверяем работу с новыми фразами
test_query = "Как восстановить доступ к личному кабинету?"
responses, similarities = assistant.find_similar_responses(test_query)

print(f"\n👨 Клиент: '{test_query}'")
print("🔍 Подходящие подсказки оператору:")
for i, (resp, sim) in enumerate(zip(responses, similarities), 1):
    print(f"{i}. [{sim:.2f}] {resp}")

Batches: 100%|██████████| 2/2 [00:00<00:00,  4.27it/s]


✅ База знаний инициализирована: 63 фраз, размерность 384, время обработки: 0.48 сек
✅ Добавлено 9 новых фраз в базу знаний

👨 Клиент: 'Как восстановить доступ к личному кабинету?'
🔍 Подходящие подсказки оператору:
1. [0.10] Для восстановления доступа к личному кабинету нажмите 'Забыли пароль' на странице входа.
2. [0.07] Вы можете заказать перевыпуск карты в личном кабинете.
3. [0.07] Актуальные акции отражены в личном кабинете.


In [74]:
# Добавляем новую категорию - ответы о кэшбэке и бонусах
cashback_phrases = [
    "Размер кэшбэка зависит от типа вашей карты.",
    "Бонусные баллы начисляются в конце каждого месяца.",
    "Вы можете потратить накопленные баллы в нашем маркетплейсе.",
    "Кэшбэк до 5% действует на категорию 'Супермаркеты'.",
    "Подробные условия бонусной программы в разделе 'Преимущества' мобильного приложения."
]

assistant.add_to_knowledge_base(cashback_phrases)

# Проверяем новую категорию
test_query = "Как работает кэшбэк?"
responses, similarities = assistant.find_similar_responses(test_query, top_k=2)

print(f"\n👨 Клиент: '{test_query}'")
print("🔍 Подходящие подсказки оператору:")
for i, (resp, sim) in enumerate(zip(responses, similarities), 1):
    print(f"{i}. [{sim:.2f}] {resp}")

✅ Добавлено 5 новых фраз в базу знаний

👨 Клиент: 'Как работает кэшбэк?'
🔍 Подходящие подсказки оператору:
1. [0.07] Для решения проблемы попробуйте очистить кэш браузера.
2. [0.06] Размер кэшбэка зависит от типа вашей карты.


In [75]:
assistant.save_knowledge_base()

✅ База знаний сохранена в директорию: knowledge_base


In [34]:
test_query = "Как я могу оплатить ежемесячный платеж по кредиту?"
responses, similarities = assistant.find_similar_responses(test_query, top_k=3)

print(f"\n👨 Клиент: '{test_query}'")
print("🔍 Подходящие подсказки оператору:")
for i, (resp, sim) in enumerate(zip(responses, similarities), 1):
    print(f"{i}. [{sim:.2f}] {resp}")


👨 Клиент: 'Как я могу оплатить ежемесячный платеж по кредиту?'
🔍 Подходящие подсказки оператору:
1. [0.07] График платежей был отправлен вам на email при оформлении кредита.
2. [0.06] Досрочное погашение кредита доступно без комиссий.
3. [0.05] Бонусные баллы начисляются в конце каждого месяца.


In [76]:
# Пример использования сохранения и загрузки
if __name__ == "__main__":
    # Создание помощника
    assistant = OperatorAssistant()
    
    # Попытка загрузить существующую базу знаний
    if not assistant.load_knowledge_base():
        # Если не удалось загрузить, инициализируем новую
        assistant.initialize_knowledge_base(knowledge_base)
        # Сохраняем для будущего использования
        assistant.save_knowledge_base()
    
    # Тестирование поиска
    test_queries = [
        "Как мне оплатить кредит?",
        "Какие у вас есть инвестиционные предложения?",
        "У меня проблема с мобильным приложением"
    ]
    
    for query in test_queries:
        print(f"\n🔍 Запрос: '{query}'")
        responses, scores = assistant.find_similar_responses(query)
        for resp, score in zip(responses, scores):
            print(f"→ {resp} (схожесть: {score:.2f})")
    
    # Добавление новых фраз и сохранение обновленной базы
    new_phrases = [
        "Для уточнения информации по вкладам нажмите 1",
        "Информацию по ипотеке можно получить в разделе 'Кредиты'"
    ]
    assistant.add_to_knowledge_base(new_phrases)
    assistant.save_knowledge_base()

✅ База знаний загружена: 77 фраз, размерность 384

🔍 Запрос: 'Как мне оплатить кредит?'
→ График платежей был отправлен вам на email при оформлении кредита. (схожесть: 0.06)
→ Досрочное погашение кредита доступно без комиссий. (схожесть: 0.06)
→ Вы обращаетесь по вопросу кредита, депозита или другого продукта? (схожесть: 0.05)

🔍 Запрос: 'Какие у вас есть инвестиционные предложения?'
→ Наши консультанты помогут вам подобрать инвестиционный продукт. (схожесть: 0.09)
→ Вы обращаетесь по вопросу кредита, депозита или другого продукта? (схожесть: 0.07)
→ Мы можем предложить вам несколько вариантов страхования. (схожесть: 0.06)

🔍 Запрос: 'У меня проблема с мобильным приложением'
→ Мобильное приложение доступно в AppStore и Google Play. (схожесть: 0.07)
→ Подробные условия бонусной программы в разделе 'Преимущества' мобильного приложения. (схожесть: 0.07)
→ Вы можете оплатить через мобильное приложение или терминал. (схожесть: 0.06)
✅ Добавлено 2 новых фраз в базу знаний
✅ База знаний сохра

In [77]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Загружаем модель
model = SentenceTransformer('./paraphrase-multilingual-MiniLM-L12-v2')

# Два слова
word1 = "Как я могу оплатить кредит?"
word2 = "График платежей был отправлен вам на email при оформлении кредита."

# Получаем эмбеддинги
embedding1 = model.encode(word1)
embedding2 = model.encode(word2)

# Считаем косинусную схожесть
similarity = cosine_similarity([embedding1], [embedding2])[0][0]
distance = 1 - similarity

print(f"Косинусное расстояние между '{word1}' и '{word2}': {distance:.4f}")

Косинусное расстояние между 'Как я могу оплатить кредит?' и 'График платежей был отправлен вам на email при оформлении кредита.': 0.4681
